# **Exploratory Analysis TCGA-BRCA Metadata**
- TCGA: The Cancer Genome Atlas  
- BRCA: Breast Invasive Carcinoma

# Import Libraries and Configurations

In [1]:
import os
import sys

import pandas as pd

# Add project root to Python's path
sys.path.append(os.path.abspath(os.path.join('..', '..')))

from config import (
    BRCA_PROCESSED_FILES_PATHS,
    BRCA_RAW_FILES_PATHS,
)

# Load and Prepare the Metadata

In [2]:
# Dataframe with cases metadata
df_cases = pd.read_csv(BRCA_PROCESSED_FILES_PATHS['cases'])

# Dataframe with cases of interest metadata
df_cases_of_interest = df_cases.query('is_case_of_interest == 1')

# Dataframe with files metadata
df_files = pd.read_csv(BRCA_PROCESSED_FILES_PATHS['files'])

# Dataframe with files of interest metadata
df_files_of_interest = df_files.query('is_file_of_interest == 1')

# DataFrame with project metadata
df_project = pd.read_csv(BRCA_RAW_FILES_PATHS['project'])

# DataFrame with files and cases
df_files_and_cases = df_files \
    .merge(
        right=df_cases,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# DataFrame with files and cases of interest
df_files_and_cases_of_interest = df_files_of_interest \
    .merge(
        right=df_cases_of_interest,
        left_on='case_id',
        right_on='case_id',
        how='inner'
    )

# Project Exploration

In [3]:
# Print the TCGA-BRCA project metadata DataFrame
pd.set_option('display.max_colwidth', 900)
df_project

,project_id,project_name,primary_site,disease_type,experimental_strategies,data_categories,case_count,file_count
0,TCGA-BRCA,Breast Invasive Carcinoma,['Breast'],"['Fibroepithelial Neoplasms', 'Adnexal and Skin Appendage Neoplasms', 'Adenomas and Adenocarcinomas', 'Cystic, Mucinous and Serous Neoplasms', 'Basal Cell Neoplasms', 'Squamous Cell Neoplasms', 'Epithelial Neoplasms, NOS', 'Complex Epithelial Neoplasms', 'Ductal and Lobular Neoplasms']","[{'file_count': 11079, 'case_count': 1095, 'experimental_strategy': 'RNA-Seq'}, {'file_count': 17049, 'case_count': 1072, 'experimental_strategy': 'WXS'}, {'file_count': 3621, 'case_count': 1079, 'experimental_strategy': 'miRNA-Seq'}, {'file_count': 12688, 'case_count': 952, 'experimental_strategy': 'WGS'}, {'file_count': 75, 'case_count': 74, 'experimental_strategy': 'ATAC-Seq'}, {'file_count': 14329, 'case_count': 1098, 'experimental_strategy': 'Genotyping Array'}, {'file_count': 3714, 'case_count': 1097, 'experimental_strategy': 'Methylation Array'}, {'file_count': 919, 'case_count': 881, 'experimental_strategy': 'Reverse Phase Protein Array'}, {'file_count': 1133, 'case_count': 1062, 'experimental_strategy': 'Diagnostic Slide'}, {'file_count': 1979, 'case_count': 1093, 'experimental_strategy': 'Tissue Slide'}]","[{'file_count': 21437, 'case_count': 1098, 'data_category': 'Simple Nucleotide Variation'}, {'file_count': 9282, 'case_count': 1098, 'data_category': 'Sequencing Reads'}, {'file_count': 5317, 'case_count': 1098, 'data_category': 'Biospecimen'}, {'file_count': 2288, 'case_count': 1098, 'data_category': 'Clinical'}, {'file_count': 14346, 'case_count': 1098, 'data_category': 'Copy Number Variation'}, {'file_count': 4876, 'case_count': 1097, 'data_category': 'Transcriptome Profiling'}, {'file_count': 3714, 'case_count': 1097, 'data_category': 'DNA Methylation'}, {'file_count': 919, 'case_count': 881, 'data_category': 'Proteome Profiling'}, {'file_count': 3128, 'case_count': 927, 'data_category': 'Somatic Structural Variation'}, {'file_count': 5772, 'case_count': 1098, 'data_category': 'Structural Variation'}]",1098,71079


# Cases Exploration

## All Cases

In [4]:
# Count the number of distinct cases
total_cases = df_cases['case_id'].nunique()
print(f'Number of distinct cases: {total_cases}')

Number of distinct cases: 1098


In [5]:
# Count the number of distinct cases with molecular subtype classification
total_cases = df_cases.query('pam50_mrna.notna()')['case_id'].nunique()
print(f'Number of distinct cases with molecular subtype classification: {total_cases}')

Number of distinct cases with molecular subtype classification: 514


In [6]:
# Count the number of cases associated with each disease type
df_cases \
    .groupby('disease_type') \
    .agg(cases = pd.NamedAgg(column='case_id', aggfunc='nunique')) \
    .sort_values(by='cases', ascending=False)

,cases
disease_type,
Ductal and Lobular Neoplasms,1054
"Cystic, Mucinous and Serous Neoplasms",16
Complex Epithelial Neoplasms,14
"Epithelial Neoplasms, NOS",5
Adenomas and Adenocarcinomas,3
Fibroepithelial Neoplasms,2
Squamous Cell Neoplasms,2
Adnexal and Skin Appendage Neoplasms,1
Basal Cell Neoplasms,1


In [7]:
# Count the number of cases associated with each molecular subtype
df_cases \
    .groupby('pam50_mrna') \
    .agg(cases = pd.NamedAgg(column='case_id', aggfunc='nunique')) \
    .sort_values(by='cases', ascending=False)

,cases
pam50_mrna,
Luminal A,231
Luminal B,127
Basal-like,98
HER2-enriched,58


In [8]:
# Count the number of cases associated with each subtype and disease
df_cases \
    .groupby(['pam50_mrna', 'disease_type']) \
    .agg(cases = pd.NamedAgg(column='case_id', aggfunc='nunique')) \
    .sort_values(by=['pam50_mrna', 'cases'])

cases
pam50_mrna    disease_type                                
Basal-like    Epithelial Neoplasms, NOS                  1
              Complex Epithelial Neoplasms               2
              Ductal and Lobular Neoplasms              95
HER2-enriched Adnexal and Skin Appendage Neoplasms       1
              Ductal and Lobular Neoplasms              57
Luminal A     Adenomas and Adenocarcinomas               1
              Epithelial Neoplasms, NOS                  1
              Squamous Cell Neoplasms                    1
              Ductal and Lobular Neoplasms             228
Luminal B     Cystic, Mucinous and Serous Neoplasms      2
              Ductal and Lobular Neoplasms             125

## Cases of Interest

In [9]:
# Count the number of distinct cases
total_cases = df_cases_of_interest['case_id'].nunique()
print(f'Number of distinct cases of interest: {total_cases}')

Number of distinct cases of interest: 488


In [10]:
# Count the number of cases associated with tumor tissue analysis
total_cases = df_cases_of_interest['has_tumor_files_of_interest'].sum()
print(f'Number of cases associated with tumor tissue analysis: {total_cases}')

Number of cases associated with tumor tissue analysis: 486


In [11]:
# Count the number of cases associated with normal tissue analysis
total_cases = df_cases_of_interest['has_normal_files_of_interest'].sum()
print(f'Number of cases associated with paired normal tissue analysis: {total_cases}')

Number of cases associated with paired normal tissue analysis: 56


In [12]:
# Count the number of cases associated with each disease type
df_cases_of_interest \
    .groupby('disease_type') \
    .agg(cases = pd.NamedAgg(column='case_id', aggfunc='nunique')) \
    .sort_values(by='cases', ascending=False)

,cases
disease_type,
Ductal and Lobular Neoplasms,488


In [13]:
# Count the number of cases associated with each molecular subtype
df_cases_of_interest \
    .groupby('pam50_mrna') \
    .agg(
        cases = pd.NamedAgg(column='case_id', aggfunc='nunique'),
        tumor_cases = pd.NamedAgg(
            column='has_tumor_files_of_interest', aggfunc='sum'
        ),
        normal_cases = pd.NamedAgg(
            column='has_normal_files_of_interest', aggfunc='sum'
        ),
    ) \
    .sort_values(by='cases', ascending=False)

,cases,tumor_cases,normal_cases
pam50_mrna,,,
Luminal A,224,223,29
Luminal B,120,120,14
Basal-like,88,87,8
HER2-enriched,56,56,5


# Files Exploration

## All Files

In [14]:
# Count the number of distinct files
total_files = df_files['file_id'].nunique()
print(f'Number of distinct files: {total_files}')

Number of distinct files: 71079


In [15]:
# Count the number of files associated with each experimental strategy
df_files \
    .groupby('experimental_strategy') \
    .agg(files = pd.NamedAgg(column='file_id', aggfunc='nunique')) \
    .sort_values(by='files', ascending=False)

,files
experimental_strategy,
WXS,17049
Genotyping Array,14329
WGS,12688
RNA-Seq,11079
Methylation Array,3714
miRNA-Seq,3621
Tissue Slide,1979
Diagnostic Slide,1133
Reverse Phase Protein Array,919


In [16]:
# Count the number of files associated with each data type
df_files \
    .groupby('data_type') \
    .agg(files = pd.NamedAgg(column='file_id', aggfunc='nunique')) \
    .sort_values(by='files', ascending=False)

,files
data_type,
Annotated Somatic Mutation,10626
Aligned Reads,9282
Raw Simple Somatic Mutation,6564
Transcript Fusion,4924
Structural Rearrangement,3976
Gene Level Copy Number,3314
Copy Number Segment,3256
Slide Image,3112
Masked Intensities,2476


In [17]:
# Count the number of data types associated with files
df_files \
    .groupby([
        'experimental_strategy', 'access', 'data_type', 'data_format',
    ]) \
    .agg(files = pd.NamedAgg(column='file_id', aggfunc='nunique')) \
    .sort_values(by=['experimental_strategy', 'access', 'data_type'])

files
experimental_strategy       access     data_type                           data_format       
ATAC-Seq                    controlled Aligned Reads                       BAM             75
Diagnostic Slide            open       Slide Image                         SVS           1133
Genotyping Array            controlled Raw Intensities                     CEL           2263
                                       Simple Germline Variation           TSV           2263
                            open       Allele-specific Copy Number Segment TXT           2144
                                       Copy Number Segment                 TXT           2229
                                       Gene Level Copy Number              TSV           3201
                                       Masked Copy Number Segment          TXT           2229
Methylation Array           open       Masked Intensities                  IDAT          2476
                                       Methylation Beta Value              TXT           1238
RNA-Seq                     controlled Aligned Reads                       BAM           3693
                                       Splice Junction Quantification      TSV           1231
                                       Transcript Fusion                   BEDPE         2462
                                                                           TSV           2462
                            open       Gene Expression Quantification      TSV           1231
Reverse Phase Protein Array open       Protein Expression Quantification   TSV            919
Tissue Slide                open       Slide Image                         SVS           1979
WGS                         controlled Aligned Reads                       BAM           2106
                                       Annotated Somatic Mutation          VCF           2050
                                       Intermediate Analysis Archive       TAR           1027
                                       Raw Simple Somatic Mutation         VCF           2276
                                       Structural Rearrangement            BEDPE         1988
                                                                           VCF           1988
                            open       Allele-specific Copy Number Segment TXT            113
                                       Copy Number Segment                 TXT           1027
                                       Gene Level Copy Number              TSV            113
WXS                         controlled Aggregated Somatic Mutation         MAF            992
                                       Aligned Reads                       BAM           2201
                                       Annotated Somatic Mutation          MAF           4288
                                                                           VCF           4288
                                       Raw Simple Somatic Mutation         VCF           4288
                            open       Masked Somatic Mutation             MAF            992
miRNA-Seq                   controlled Aligned Reads                       BAM           1207
                            open       Isoform Expression Quantification   TXT           1207
                                       miRNA Expression Quantification     TXT           1207

In [18]:
# Count the number of files associated with each subtype and experimental strategy
df_files_and_cases \
    .groupby(['pam50_mrna', 'experimental_strategy']) \
    .agg(files = pd.NamedAgg(column='file_id', aggfunc='nunique')) \
    .sort_values(by=['pam50_mrna', 'experimental_strategy'])

files
pam50_mrna    experimental_strategy             
Basal-like    ATAC-Seq                        12
              Diagnostic Slide                98
              Genotyping Array              1288
              Methylation Array              339
              RNA-Seq                       1017
              Reverse Phase Protein Array     88
              Tissue Slide                   212
              WGS                           1518
              WXS                           1578
              miRNA-Seq                      318
HER2-enriched ATAC-Seq                        13
              Diagnostic Slide                56
              Genotyping Array               740
              Methylation Array              189
              RNA-Seq                        567
              Reverse Phase Protein Array     53
              Tissue Slide                   122
              WGS                            902
              WXS                            898
              miRNA-Seq                      186
Luminal A     ATAC-Seq                        25
              Diagnostic Slide               234
              Genotyping Array              3046
              Methylation Array              813
              RNA-Seq                       2430
              Reverse Phase Protein Array    182
              Tissue Slide                   517
              WGS                           2697
              WXS                           3436
              miRNA-Seq                      780
Luminal B     ATAC-Seq                        15
              Diagnostic Slide               129
              Genotyping Array              1665
              Methylation Array              432
              RNA-Seq                       1287
              Reverse Phase Protein Array    106
              Tissue Slide                   277
              WGS                           1476
              WXS                           1943
              miRNA-Seq                      414

## Files of Interest

In [19]:
# Count the number of distinct files
total_files = df_files_of_interest['file_id'].nunique()
print(f'Number of distinct files: {total_files}')

Number of distinct files: 1084


In [20]:
# Count the number of files associated with tumor tissue analysis
total_files = df_files_of_interest['is_tumor_file_of_interest'].sum()
print(f'Number of files associated with tumor tissue analysis: {total_files}')

Number of files associated with tumor tissue analysis: 972


In [21]:
# Count the number of files associated with normal tissue analysis
total_files = df_files_of_interest['is_normal_file_of_interest'].sum()
print(f'Number of files associated with paired normal tissue analysis: {total_files}')

Number of files associated with paired normal tissue analysis: 112


In [22]:
# Count the number of files associated with each experimental strategy
df_files_and_cases_of_interest \
    .groupby(['experimental_strategy', 'data_type']) \
    .agg(
        files = pd.NamedAgg(column='file_id', aggfunc='nunique'),
        tumor_files = pd.NamedAgg(
            column='is_tumor_file_of_interest', aggfunc='sum'
        ),
        normal_files = pd.NamedAgg(
            column='is_normal_file_of_interest', aggfunc='sum'
        ),
    ) \
    .sort_values(by=['experimental_strategy', 'files'], ascending=False)

,,files,tumor_files,normal_files
experimental_strategy,data_type,,,
miRNA-Seq,Isoform Expression Quantification,542,486,56
RNA-Seq,Gene Expression Quantification,542,486,56


In [23]:
# Count the number of files associated with each subtype and experimental strategy
df_files_and_cases_of_interest \
    .groupby(['pam50_mrna', 'experimental_strategy']) \
    .agg(
        files = pd.NamedAgg(column='file_id', aggfunc='nunique'),
        tumor_files = pd.NamedAgg(
            column='is_tumor_file_of_interest', aggfunc='sum'
        ),
        normal_files = pd.NamedAgg(
            column='is_normal_file_of_interest', aggfunc='sum'
        ),
    ) \
    .sort_values(by=['files', 'pam50_mrna'], ascending=False)

files  tumor_files  normal_files
pam50_mrna    experimental_strategy                                  
Luminal A     RNA-Seq                  252          223            29
              miRNA-Seq                252          223            29
Luminal B     RNA-Seq                  134          120            14
              miRNA-Seq                134          120            14
Basal-like    RNA-Seq                   95           87             8
              miRNA-Seq                 95           87             8
HER2-enriched RNA-Seq                   61           56             5
              miRNA-Seq                 61           56             5

In [24]:
# Count the number of data types associated with files
df_files_of_interest \
    .groupby([
        'experimental_strategy', 'access', 'data_type', 'data_format',
    ]) \
    .agg(files = pd.NamedAgg(column='file_id', aggfunc='nunique')) \
    .sort_values(by=['experimental_strategy', 'files'],ascending=False)

,,,,files
experimental_strategy,access,data_type,data_format,
miRNA-Seq,open,Isoform Expression Quantification,TXT,542
RNA-Seq,open,Gene Expression Quantification,TSV,542
